### Урок 6. Задача lookalike (Positive Unlabeled Learning)

### Домашнее задание

1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)
3. сделать feature engineering
4. обучить любой классификатор (какой вам нравится)
5. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
6. применить random negative sampling для построения классификатора в новых условиях
7. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)
8. поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

In [36]:
# https://archive.ics.uci.edu/ml/datasets/Bank+Marketing

In [37]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score


def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 
    
    return f1, roc, rec, prc


res = [] # общие результаты

df = pd.read_csv("bank-full.csv", delimiter=";")
df.head(3)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no


In [38]:
print(df.shape)

(45211, 17)


In [39]:
df.iloc[:, -1].value_counts()

no     39922
yes     5289
Name: y, dtype: int64

In [40]:
df['y'] = df['y'].map({'yes': 1, 'no': 0})
df = pd.get_dummies(df)

Разбиваем выборку на тренировочную и тестовую части и обучаем модель (градиентный бустинг)

In [41]:
x_data = df.copy()
y_data = x_data.pop('y')

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

In [42]:
model = xgb.XGBClassifier()

model.fit(x_train, y_train)
y_predict = model.predict(x_test)

In [43]:
res.append(['pure target', *evaluate_results(y_test, y_predict)])

Classification results:
f1: 55.04%
roc: 72.54%
recall: 48.62%
precision: 63.41%


### Теперь очередь за PU learning

Представим, что нам неизвестны негативы и часть позитивов

In [44]:
mod_data = df.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data['y'].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 1323/5289 as positives and unlabeling the rest


Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)

In [45]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample, 'class_test'] = 1
print('target variable:\n', mod_data['class_test'].value_counts())

target variable:
 -1    43888
 1     1323
Name: class_test, dtype: int64


In [46]:
mod_data.head(3)

,age,balance,day,duration,campaign,pdays,previous,y,job_admin.,job_blue-collar,...,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown,class_test
0,58,2143,5,261,1,-1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,-1
1,44,29,5,151,1,-1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,-1
2,33,2,5,76,1,-1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,-1


Remember that this data frame (x_data) includes the former target variable that we keep here just to compare the results

[:-2] is the original class label for positive and negative data [:-1] is the new class for positive and unlabeled data

In [47]:
x_data = mod_data.drop(columns=['y', 'class_test']).values # just the X 
y_labeled = mod_data['class_test'].values # new class (just the P & U)
y_positive = mod_data['y'].values # original class

### 1. random negative sampling

In [48]:
mod_data = mod_data.sample(frac=1)

data_N = mod_data[mod_data['class_test'] == -1] 
data_P = mod_data[mod_data['class_test'] == 1] 

neg_sample = data_N[:data_P.shape[0]]
sample_test = data_N[data_P.shape[0]:]
pos_sample = data_P.copy() 

print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(1323, 53) (1323, 53)


In [49]:
model = xgb.XGBClassifier()
sample_train.loc[sample_train['class_test']==-1, 'class_test'] = 0 # потому что XGBClassifier обучается на 0/1, а у нас -1/1

model.fit(sample_train.drop(columns=['y', 'class_test']).values, 
          sample_train['class_test'].values)
y_predict = model.predict(sample_test.drop(columns=['y', 'class_test']).values)
res.append(['random negative sampling', *evaluate_results(sample_test['y'].values, y_predict)])

Classification results:
f1: 48.34%
roc: 84.08%
recall: 84.68%
precision: 33.83%


In [50]:
pd.DataFrame(res, columns=['method', 'f1', 'roc', 'recall', 'precision'])

,method,f1,roc,recall,precision
0,pure target,0.550418,0.725358,0.486220,0.634146
1,random negative sampling,0.483427,0.840850,0.846812,0.338269


In [ ]:
# улучшился отсев неверных результатов. Но точность нахождения верных упала